## **Detección de Ataques en Tráfico de Red**
El presente proyecto utiliza el dataset [Cybersecurity 🪪 Intrusion 🦠 Detection Dataset](https://www.kaggle.com/datasets/dnkumars/cybersecurity-intrusion-detection-dataset) para entrenar un modelo de clasificación que permite identificar si un paquete determinado es malicioso o no.

...


### **1.- Importar Dataset y Librerías Necesarias** 
El dataset se descargó desde [Kaggle](https://www.kaggle.com/datasets/dnkumars/cybersecurity-intrusion-detection-dataset) y se incluyó dentro de assets con el nombre de cybersecurity_data.csv. Los datos del archivo csv se importan creando un dataset para explorar la información

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

path= os.path.join(".", "assets", "cybersecurity_data.csv")
df_ids=pd.read_csv(path, sep=",") 
display(df_ids.head(2))
display(df_ids.tail(2))
display(len(df_ids))

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
9535,SID_09536,406,TCP,4,86.664703,AES,0.537417,1,Chrome,1,0
9536,SID_09537,340,TCP,6,86.876744,NaN,0.277069,4,Chrome,1,1


9537

### **2.- Exploración de los datos**
El dataframe tiene un total de 9537 registros, en donde los índices van desde 0 a 9536. A continuación se explora los datos para identificar el tipo de datos y la existencia de valores nulos.

In [2]:
display(df_ids.describe())
display(df_ids.info())
nulos= df_ids.isnull().sum()
display(nulos)


,network_packet_size,login_attempts,session_duration,ip_reputation_score,failed_logins,unusual_time_access,attack_detected
count,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000
mean,500.430639,4.032086,792.745312,0.331338,1.517773,0.149942,0.447101
std,198.379364,1.963012,786.560144,0.177175,1.033988,0.357034,0.497220
min,64.000000,1.000000,0.500000,0.002497,0.000000,0.000000,0.000000
25%,365.000000,3.000000,231.953006,0.191946,1.000000,0.000000,0.000000
50%,499.000000,4.000000,556.277457,0.314778,1.000000,0.000000,0.000000
75%,635.000000,5.000000,1105.380602,0.453388,2.000000,0.000000,1.000000
max,1285.000000,13.000000,7190.392213,0.924299,5.000000,1.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9537 entries, 0 to 9536
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   session_id           9537 non-null   object 
 1   network_packet_size  9537 non-null   int64  
 2   protocol_type        9537 non-null   object 
 3   login_attempts       9537 non-null   int64  
 4   session_duration     9537 non-null   float64
 5   encryption_used      7571 non-null   object 
 6   ip_reputation_score  9537 non-null   float64
 7   failed_logins        9537 non-null   int64  
 8   browser_type         9537 non-null   object 
 9   unusual_time_access  9537 non-null   int64  
 10  attack_detected      9537 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 819.7+ KB


None

session_id                0
network_packet_size       0
protocol_type             0
login_attempts            0
session_duration          0
encryption_used        1966
ip_reputation_score       0
failed_logins             0
browser_type              0
unusual_time_access       0
attack_detected           0
dtype: int64

### **3.- Tratamiento de los Datos**
La columna encryption_used es la única que tiene valores nulos. Al revisar los datos únicos de la columna encryption_used  y la documentación de origen del dataset, se identifica que los valores nulos en esta columna corresponden a comunicaciones sin cifrado, por lo que se reemplazan los nulos por la cadena "TEXT", que representará las comunicaciones en texto plano.

En la exploración del tipo de datos se identifica que las columnas "session", "protocolo", "encryption", y "browser" no tienen valores numéricos. En estas columnas se revisan sus valores únicos para determinar la viabilidad de codificarlas como variables categóricas. 

Para el id se utiliza len con la finalidad de identificar posibles valores duplicados. Si el valor de len de esta es menor que la cantidad de registros se determinaría que existen sesiones duplicadas. En la exploración se identifica que este no es el caso. Para las columnas complementarias, basados en la documentación se realiza una exploración de sus valores únicos para codificarlos de manera posterior.

In [3]:
encryptions=df_ids["encryption_used"].unique()
display(encryptions)
df_ids["encryption_used"]=df_ids["encryption_used"].fillna("TEXT")
display(len(df_ids["session_id"].unique()))
display(df_ids["protocol_type"].unique())
display(df_ids["encryption_used"].unique())
display(df_ids["browser_type"].unique())

array(['DES', 'AES', nan], dtype=object)

9537

array(['TCP', 'UDP', 'ICMP'], dtype=object)

array(['DES', 'AES', 'TEXT'], dtype=object)

array(['Edge', 'Firefox', 'Chrome', 'Unknown', 'Safari'], dtype=object)

## KneighborsClasifier
## Tree Classifier
## Regresión Logística --> Normalizar los datos standar scaler

### **4.- División de Datos en Bloques**
Una vez realizado el tratamiento de los datos, se procede a dividir el dataset en tres partes que serán utilizadas para entrenamiento, validación y testing.

In [5]:
train=df_ids[:9537//3]
validate=df_ids[9537//3:2*9537//3]
testing=df_ids[2*9537//3:]
display(train.head(1))
display(train.tail(1))
display(validate.head(1))
display(validate.tail(1))
display(testing.head(1))
display(testing.tail(1))

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
3178,SID_03179,695,TCP,2,523.488907,AES,0.566019,4,Chrome,0,1


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
3179,SID_03180,603,UDP,4,1053.525126,TEXT,0.398444,0,Edge,0,0


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
6357,SID_06358,191,UDP,2,998.237454,AES,0.279871,3,Chrome,1,1


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
6358,SID_06359,103,TCP,2,63.786486,AES,0.421514,2,Unknown,0,0


,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
9536,SID_09537,340,TCP,6,86.876744,TEXT,0.277069,4,Chrome,1,1
